In [79]:
import os
import pandas as pd
import numpy as np
import csv
import openpyxl

In [64]:
class data_of_student:
    
    def __init__(self,roll_no,name,email,aemail,contact):
        self.name=name
        self.email=email
        self.aemail=aemail
        self.contact=contact
        self.roll_no=roll_no
        self.courses=dict()
    
# course obj will ltp and other info of a course 
class course:
    def __init__(self,lec,tut,prac):
        self.lec=lec
        self.tut=tut
        self.prac=prac
        
class course_info_of_student:
    
    def __init__(self,lval,tval,pval):
        
        # here if lval=0 then there is no need to check if the student has given feedback so we initialize it to True  
        # if lval!=0 then we initialize it to False and then we check using cfbs similarly for tval and pval 
        
        self.lval=not lval 
        self.tval=not tval
        self.pval=not pval
        self.reg_sem=''
        self.sched_sem=''
        

In [77]:
def feedback_not_submitted():


    #ltp_mapping_feedback_type = {1: 'lecture', 2: 'tutorial', 3:'practical'}
    
    rows=[]
    
    # checking if the student has given feedback of ltp of all the courses that they registered.
    for stud in student:
        s=student[stud]   
        for cour in s.courses:
            c=s.courses[cour]
            
            # here if all feedbacks that are should be given so, lval, tval, pval should be true 
            #for the course else they should be added to the rows with thier data and course name
            if not(c.lval and c.tval and c.pval):
                
                eachrow=[stud,c.reg_sem,c.sched_sem,cour,s.name,s.email,s.aemail,s.contact]
                rows.append([])
                
                for x in eachrow:
                    rows[-1].append(x)
                    
                break    
    
    data=np.array(rows)
    df=pd.DataFrame(data,columns=['rollno','register_sem','schedule_sem','subno','Name','email','aemail','contact'])
    
    output_file_name='course_feedback_remaining.xlsx'
    if os.path.exists(output_file_name):
        os.remove(output_file_name)
    df.to_excel(output_file_name) 
    
 

In [66]:
 #courses data and their ltp
cmdoe=pd.read_csv('course_master_dont_open_in_excel.csv')

#student details 
s_info=pd.read_csv('studentinfo.csv')   

# to know what courses are taken by students
s_data=pd.read_csv('course_registered_by_all_students.csv')

#feedback form
cfbs=pd.read_csv('course_feedback_submitted_by_students.csv') 


In [67]:
# course_info dict will store objects of courses with key as course number
course_info=dict()

#student dict store objects of data_of_student with key as roll number
student=dict()

In [68]:
# fetching data from cmdoe which has courses and their info 
for index, x in cmdoe.iterrows():
    
    l,t,p=map(float,list(x['ltp'].split('-')))
    course_info[x['subno']]=course(l,t,p)

In [69]:
# fetching data from s_info which has all students information
for index, name in s_info.iterrows():
    
    if name['Roll No'] not in student:
        
        student[name['Roll No']]=data_of_student(name['Roll No'],name['Name'],name['email'],name['aemail'],name['contact'])


In [71]:
# storing what courses were taken by each student 
for index, x in s_data.iterrows():
    
    # if the student is not present in student info
    if x['rollno'] not in student:
        student[x['rollno']]=data_of_student(x['rollno'],'Not_in_student_info','Not_in_student_info','Not_in_student_info','Not_in_student_info')
    
    
    # storing course_info_of_student in courses of data_of_student object of student[x['rollno']]
    lval=int(course_info[x['subno']].lec)
    tval=int(course_info[x['subno']].tut)
    pval=int(course_info[x['subno']].prac)
    
    student[x['rollno']].courses[x['subno']]=course_info_of_student(lval,tval,pval)
    
    
    #similarly 
    student[x['rollno']].courses[x['subno']].reg_sem=x['register_sem']
    student[x['rollno']].courses[x['subno']].sched_sem=x['schedule_sem']
        

In [75]:
for index, i in cfbs.iterrows():
    
    if i['feedback_type']==1 :
        student[i['stud_roll']].courses[i['course_code']].lval=1
    elif i['feedback_type']==2:
        student[i['stud_roll']].courses[i['course_code']].tval=1
    else:
        student[i['stud_roll']].courses[i['course_code']].pval=1


In [78]:
feedback_not_submitted()